In [9]:
import numpy as np
import pandas as pd
from numpy.random import uniform, normal  
from scipy.optimize import Bounds
from scipy.optimize import minimize
import matplotlib.pyplot as plt 

In [10]:
# k: the number of studies
k_list = [10,30,50,100]
# each element of `parameter_constellation` has the form (mu, sigma^2,tau^2) 
# where tau^2 is the between-study variance, sigma^2*u_i is the within-study variance
parameter_constellation = [(0,12,4),(0,9,4),(0,4,4),(0,2,6)]
# heterogeniety measure: J^2 = tau^2/(tau^2+sigma^2)
J_square_list = [params[2]/(params[2]+params[1]) for params in parameter_constellation]
print('J square =', J_square_list)

J square = [0.25, 0.3076923076923077, 0.5, 0.75]


In [3]:
# normalised version, average over k studies (using np.mean rather than np.sum in nll and jac)
# use softplus rather than square to make sure sigma^2, tau^2 > 0, because softplus grows linearly.

def softplus(x):
    return np.log(np.exp(x) + 1.0)

def d_softplus(x):
    return 1. - 1./(np.exp(x) + 1.0)

def inv_softplus(x):
    return np.log(np.exp(x) - 1.0)

def nll(x, D, u):
    # x = np.array([mu, sigma, tau])
    mu, sigma_square, tau_square = x[0], softplus(x[1]), softplus(x[2])
    return 1/2*np.mean((D - mu)**2/(tau_square + sigma_square*u) \
            +np.log(tau_square + sigma_square*u)+np.log(2*np.pi))  

# Jacobian: the matrix of all its first-order partial derivatives
def jac(x, D, u):
    mu, sigma_square, tau_square = x[0], softplus(x[1]), softplus(x[2])
    a = D - mu
    b = tau_square + sigma_square * u
    # convert the derivative with respect to sigma/tau, instead of sigma^2/tau^2
    d_mu = np.mean(a / b)
    d_sigma_square = (np.mean((a**2 * u) / b**2) - np.mean(u / b))
    d_tau_square = (np.mean(a**2 / b**2) - np.mean(1./ b))
    d0 = d_mu
    d1 = d_sigma_square * d_softplus(x[1])
    d2 = d_tau_square * d_softplus(x[2])
    return - np.array([d0, d1, d2]) 



# mse(theta_hat, theta) = Var(theta_hat) + bias(theta_hat, theta)^2
def mse(param_ast, param):
    bias = np.mean(param_ast)-param
    mse = np.var(param_ast) + bias**2
    return mse

# summarize all the needed statistical infomation: [mean, bias, standard_deviation]
def stats_info(param_ast, param):
    result_list = [np.mean(param_ast)-param, np.std(param_ast), mse(param_ast,param)]
    return result_list

In [4]:
class Replication(object):
    
    def __init__(self, k, mu, sigma_square, tau_square):
        self.k = k 
        self.mu_gt = mu 
        self.sigma_square_gt = sigma_square 
        self.tau_square_gt = tau_square 
        self.u = uniform(0.02, 0.2, self.k)
        self.x = normal(self.mu_gt, self.tau_square_gt**0.5, self.k)
        self.D = normal(self.x, np.sqrt(self.sigma_square_gt)*self.u**0.5, self.k)
        
    def optimize(self, method="UC"):
        # random but reasonable initial point
        mu = uniform(-2.0, 2.0)
        sigma = uniform(0., 15.)
        tau = uniform(0., 8.)
        # the initialisation does not look reasonable, how about (mu, 1., 1.)
        if method == "UC":
            self.minimizer = minimize(nll, np.array([mu, 1.0, 1.0]), \
                            args=(self.D, self.u), method="BFGS", jac=jac, tol=1e-20, options={"disp":False, "maxiter":100})
    
    def query(self, what):
        if what == "parameters":
            params = self.minimizer.x
            params = [params[0], softplus(params[1]), softplus(params[2])]
            return params
        elif what == "status":
            return self.minimizer.status
        elif what == "jacobian":
            return self.minimizer.jac
        elif what == "curvature":
            return np.linalg.det(self.minimizer.hess_inv)
        

In [5]:
def simulate(method, k, mu, sigma_square, tau_square, num_replications=10):
    replications = [Replication(k, mu,sigma_square,tau_square) for i in range(num_replications)]
    [r.optimize(method=method) for r in replications]
    return replications

def stats(replications):
    all_params = np.array([r.query("parameters") for r in replications])
    all_mu_ast = all_params[:, 0]
    all_sigma_square_ast = all_params[:, 1]
    all_tau_square_ast = all_params[:, 2]
    # return the result for mu/sigma/tau in the form of: [mean, bias, std]
    mu_info = stats_info(all_mu_ast, replications[0].mu_gt)
    sigma_square_info = stats_info(all_sigma_square_ast, replications[0].sigma_square_gt)
    tau_square_info = stats_info(all_tau_square_ast, replications[0].tau_square_gt)
    return mu_info, sigma_square_info, tau_square_info

In [11]:
np.random.seed(234)
method = "UC"
for idx, params in enumerate(parameter_constellation):
    mu, sigma_square, tau_square = params
    for k_idx, k in enumerate(k_list):
        rs = simulate(method, k, mu, sigma_square, tau_square, num_replications=1000)
        print("k={0}, mu={1}, sigma_square={2}, tau_square={3}".format(k, mu, sigma_square, tau_square))
        s = stats(rs)
        print("mu:", s[0])
        print("sigma_square:", s[1])
        print("tau_square:", s[2])

k=10, mu=0, sigma_square=12, tau_square=4
mu: [-0.02992544041330998, 0.7465887754069773, 0.5582903315476205]
sigma_square: [4.874877258485643, 23.137003937747384, 559.0853795006384]
tau_square: [-1.1093537708971306, 2.6371588231745138, 8.185272447650869]
k=30, mu=0, sigma_square=12, tau_square=4
mu: [0.0014043449564344924, 0.4188495309974035, 0.17543690180150157]
sigma_square: [5.181298014446519, 19.271822870727217, 398.249005875192]
tau_square: [-0.6877819227420341, 2.114482992305348, 4.944082297999307]
k=50, mu=0, sigma_square=12, tau_square=4
mu: [-0.0005853540021769299, 0.33516705191628493, 0.1123372953295615]
sigma_square: [2.9041804710737047, 15.878508161258, 260.56128563570286]
tau_square: [-0.42071006588677484, 1.7780388014758357, 3.338418939092081]
k=100, mu=0, sigma_square=12, tau_square=4
mu: [-0.006220266099015161, 0.2311500104548038, 0.05346901904359846]
sigma_square: [1.8740721420754003, 12.547166201979685, 160.94352609380442]
tau_square: [-0.25482701884884307, 1.39826880

In [12]:
from wxpy import *

In [13]:
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 余一帆


In [27]:
jin = bot.friends().search(u'徐进')[0]

In [30]:
for i in range(10):
    jin.send('想你啦我的kk')

LOG OUT!
